In [1]:
!pip install transformers torch

In [2]:
# 2. Подключение Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# 3. Импорт необходимых модулей
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# 4. Путь к модели на Google Drive
model_path = "/content/drive/MyDrive/gemmaX2/"

# 5. Загрузка токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,  # Используем 16-bit точность для экономии памяти
    device_map="auto"           # Автоматически распределяем модель по доступным устройствам
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
# 6. Переводим модель в режим оценки
model.eval()

def generate_response(prompt, max_new_tokens=256):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            repetition_penalty=1.2,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [77]:
def generate_hse_answer(question: str, context: str):
  system_prompt = """
  Ты - система информационного поиска НИУ ВШЭ.
  Создай краткий ответ на основе предоставленных документов.
  Используй только информацию из источников.
  Отвечай на русском языке, не повторяйся.
  """

  question = "На каком курсе начинается военная кафедра и в какой день она обычно проходит?"
  contexst = """[Document(page_content=' в ходе которого студенты\\nприводятся к военной присяге. После учебного сбора проводится итоговая\\nаттестация. Студентам, успешно прошедшим военную подготовку, по окончании\\nуниверситета присваивается воинское звание сержант и они зачисляются в запас.\\nДля тех, кто изъявит желание пройти военную подготовку, в конце 1 курса\\nобучения в университете (в апреле – мае) в военном учебном центре проводится\\nотбор на конкурсной основе.', metadata={'download_url': 'https:\/\/spb.hse.ru\/milspb\/', 'file_name': 'Военный учебный центр НИУ ВШЭ в СПБ.docx', 'source': 'внешний сайт', 'topic_tag_1': 'Учебный процесс', 'topic_tag_2': 'ВУЦ', 'topic_tag_3': 'Другое', 'url': 'https:\/\/spb.hse.ru\/milspb\/', 'user_tag_1': 'Санкт-Петербург', 'user_tag_2': 'бакалавриат', 'user_tag_3': 'аспирантура', 'user_tag_4': 'магистратура', 'user_tag_5': 'специалитет'}), Document(page_content='Военный учебный центр (Нижний Новгород)\\n[О военном учебном центре](https:\/\/nnov.hse.ru\/milit\/about) Военная подготовка\\nв университете началась в 2002 году в соответствии с постановлением\\nПравительства Российской Федерации. Студенты получили возможность в\\nдобровольном порядке проходить курс военной подготовки. Срок обучения по\\nпрограмме подготовки – 1,5 года. После успешного завершения военных сборов в\\nвойсках, принятия Присяги и получения диплома об окончании университета\\nвыпускники Военного учебного центра зачисляются в запас без прохождения службы\\nпо призыву. В соответствии с постановлением Правительства Российской Федерации\\nот 8 мая 2002 года № 296 была образована военная кафедра при Государственном\\nуниверситете – Высшей школе экономики. В 2009 году ГУ-ВШЭ получил новый статус\\nи наименование – Национальный исследовательский университет «Высшая школа\\nэкономики» (НИУ ВШЭ). Распоряжением Правительства Российской Федерации от\\n13.03.2019 N 427-р \"О военных учебных центрах при федеральных государственных\\nобразовательных организациях высшего образования\" п. 42, военная кафедра при\\nНИУ ВШЭ была преобразована в Военный учебный центр при федеральном', metadata={'download_url': 'https:\/\/nnov.hse.ru\/milit\/about', 'file_name': 'Военный учебный центр (Нижний Новгород).docx', 'source': 'внешний сайт', 'topic_tag_1': 'Учебный процесс', 'topic_tag_2': 'ВУЦ', 'topic_tag_3': 'Другое', 'url': 'https:\/\/nnov.hse.ru\/milit\/about', 'user_tag_1': 'Нижний Новгород', 'user_tag_2': 'бакалавриат', 'user_tag_3': 'аспирантура', 'user_tag_4': 'магистратура', 'user_tag_5': 'специалитет'}), Document(page_content='луатация и ремонт радиоэлектронного оборудования самолетов, вертолётов и\\nавиационных ракет 751100 — Защита информационных технологий И по военно-\\nучетным специальностям сержантов,солдат: 100182 — Стрелковые, командир\\nстрелкового отделения 106646-543 — Разведывательные, разведчик-оператор СБР,\\nПСНР В военном учебном центре создано 2 кафедры: Кафедра Сухопутных войск -\\nцикл войсковой разведки - цикл подготовки сержантов - цикл подготовки солдат\\nКафедра Воздушно-космических сил - цикл РКО - цикл ВВС 2 отдельных цикла: -\\nЦикл Ракетных войск стратегического назначения \\\\- Цикл общевоенной подготовки\\nВоенный учебный центр размещен на 4-х этажах 5-ти этажного здания и в своём\\nсоставе имеет 28 учебных аудиторий, преподавательские кабинеты и технические\\nпомещения (общая площадь составляет более 3500 кв.м.). Во внутреннем дворе\\nразмещен строевой плац более 200 кв.м.Сейчас военную подготовку в ВУЦ проходят\\nсвыше 900 студентов университета (в том числе и филиалов). За этот период\\nвоенный учебный', metadata={'download_url': 'https:\/\/nnov.hse.ru\/milit\/about', 'file_name': 'Военный учебный центр (Нижний Новгород).docx', 'source': 'внешний сайт', 'topic_tag_1': 'Учебный процесс', 'topic_tag_2': 'ВУЦ', 'topic_tag_3': 'Другое', 'url': 'https:\/\/nnov.hse.ru\/milit\/about', 'user_tag_1': 'Нижний Новгород', 'user_tag_2': 'бакалавриат', 'user_tag_3': 'аспирантура', 'user_tag_4': 'магистратура', 'user_tag_5': 'специалитет'}), Document(page_content=' и другие преподаватели этой кафедры (цикла). Из состава комиссии начальник кафедры (цикла) назначает председателя комиссии.\\n4.11. Оценка за зачет (экзамен) в рамках второй пересдачи выставляется по согласованию членов комиссии. При расхождении оценок решающей является оценка председателя комиссии.\\n4.12. По результатам второй пересдачи оформляется ведомость, которая подписывается всеми членами комиссии.\\n4.13. Расписание пересдач утверждает начальник Военного учебного центра или начальник кафедры по каждому направлению военной подготовки.\\n4.14. Началом периода пересдач считается следующая неделя после проведения зачета (экзамена) и доведения оценки до студентов.\\n4.15. Периоды пересдач не могут завершаться позднее сроков, указанных в приложении.\\n4.16. Расписание пересдач включает как минимум две даты приема первых пересдач зачета (экзамена) и одну дату приема второй пересдачи (с комиссией) по каждой дисциплине (с промежутком между каждой пересдачей не менее одной недели).\\n4.17. Утвержденные даты пересдач доводятся до сведения студентов в день объявления оценок за зачет или экзамен.\\n4.18. Студент, имеющий задолженность или пропустивший зачет (экзамен', metadata={'additional_info': 'Положение о текущем контроле успеваемости и промежуточной аттестации студентов, обучающихся в Военном учебном центре Национального исследовательского университета «Высшая школа экономики»', 'download_url': 'https:\/\/www.hse.ru\/data\/xf\/110\/390\/1716\/6.18.1-01_151222-1 Итоговый документ.docx', 'file_name': 'document_164.docx', 'source': 'положения и иные документы', 'topic_tag_1': 'Учебный процесс', 'topic_tag_2': 'ВУЦ', 'topic_tag_3': 'Другое', 'url': 'https:\/\/www.hse.ru\/docs\/802687500.html', 'user_tag_1': 'Пермь', 'user_tag_2': 'Санкт-Петербург', 'user_tag_3': 'Москва', 'user_tag_4': 'Нижний Новгород', 'user_tag_5': 'бакалавриат', 'user_tag_6': 'специалитет'})]"""

  user_query = f"""
  {question}
  =========
  {contexst}
  """

  full_prompt = f"{system_prompt}\nВопрос: {user_query}\nОтвет:"
  response = generate_response(full_prompt)
  return(response.split("Ответ:")[-1].strip())

In [78]:
context = """[Document(page_content='ающихся)\\nВ соответствии с частью 4 статьи 34 Федерального закона от 29.12.2012 №273-ФЗ «Об образовании в Российской Федерации» НИУ ВШЭ обеспечивает обучающимся реализацию их права на посещение по своему выбору мероприятий, которые проводятся в университете и не предусмотрены учебным планом.\\nК числу мероприятий, не предусмотренных учебным планом, относятся праздники, конкурсы, тематические вечера, фестивали, спортивные соревнования, выставки и другие мероприятия, порядок проведения которых не установлен иными локальными нормативными актами НИУ ВШЭ.\\nМероприятия, не предусмотренные учебным планом, проводятся в целях:\\n1.4.1. личностного развития обучающихся, формирования профессиональной культуры будущих специалистов, их гражданской позиции, мировоззрения и социальной активности;\\n1.4.2. реализации гражданско-патриотического, эстетического, духовного физического воспитания и профессиональной ориентации обучающихся;\\n1.4.3. организации активного отдыха обучающихся и создания условий для наиболее полного раскрытия их творческого потенциала;\\n1.4.4. удовлетворения познавательных интересов обучающихся, выходящих за рамки выбранной ими образовательной программы.\\nЗадачами мероприятий являются:\\nсоздание условий для реализации способностей, склонностей, интересов обучающихся', metadata={'additional_info': 'Порядок посещения обучающимися Национального исследовательского университета «Высшая школа экономики» по своему выбору мероприятий, проводимых в университете и не предусмотренных учебным планом', 'download_url': 'https:\/\/www.hse.ru\/data\/xf\/604\/933\/1204\/6.18-01_070624-8 Итоговый документ.docx', 'file_name': 'document_232.docx', 'source': 'положения и иные документы', 'topic_tag_1': 'Учебный процесс', 'topic_tag_2': 'Внеучебка', 'topic_tag_3': 'Другое', 'url': 'https:\/\/www.hse.ru\/docs\/239587638.html', 'user_tag_1': 'Пермь', 'user_tag_2': 'Санкт-Петербург', 'user_tag_3': 'Москва', 'user_tag_4': 'Нижний Новгород', 'user_tag_5': 'бакалавриат', 'user_tag_6': 'специалитет', 'user_tag_7': 'магистратура', 'user_tag_8': 'аспирантура'}), Document(page_content=' Читай новости и материалы о разных сферах\\nвнеучебки. Наука, спорт, социальные проекты, карьера, предпринимательство,\\nкультура и психологическое благополучие. Обо всем этом в наших социальных\\nсетях: [@extrahse](https:\/\/vk.com\/extrahse) [@extrahse](https:\/\/t.me\/extrahse)\\nХочешь стать частью нашей команды? Действуй! Мы всегда рады\\nэкстраординарным и активным студентам Наша рассылка. Только самые важные\\nновости и мероприятия [Подписаться](https:\/\/extra.hse.ru\/polls\/893843165.html)\\nКонтакты Покровский бульвар, д. 11\\n[extra@hse.ru](https:\/\/www.hse.rumailto:extra@hse.ru)', metadata={'download_url': 'https:\/\/extra.hse.ru\/#directions', 'file_name': 'Студенческие организации.docx', 'source': 'внешний сайт', 'topic_tag_1': 'Внеучебка', 'topic_tag_2': 'Другое', 'url': 'https:\/\/extra.hse.ru\/#directions', 'user_tag_1': 'Москва', 'user_tag_2': 'бакалавриат', 'user_tag_3': 'аспирантура', 'user_tag_4': 'магистратура', 'user_tag_5': 'специалитет'}), Document(page_content=' студентов? Участие в студенческом самоуправлении\\nдает студентам возможность не только выразить свое мнение по поводу того или\\nиного вопроса, но и проявить инициативу по улучшению жизни студентов в стенах\\nуниверситета. Это работает не только в масштабах всего университета -\\nстудсоветы общежитий ежедневно помогают множеству студентов решать бытовые\\nпроблемы разного масштаба, а советы факультетов занимаются более локальными\\nпроблемами студентов, связанными не только с учебным процессом, но и,\\nнапример, с комфортом обучения в том или ином корпусе.', metadata={'download_url': 'http:\/\/students.perm.hse.ru\/studsovet\/faq', 'file_name': 'Студенческий совет НИУ ВШЭ – Пермь.docx', 'source': 'внешний сайт', 'topic_tag_1': 'Внеучебка', 'topic_tag_2': 'Другое', 'url': 'http:\/\/students.perm.hse.ru\/studsovet\/faq', 'user_tag_1': 'Пермь', 'user_tag_2': 'бакалавриат', 'user_tag_3': 'магистратура', 'user_tag_4': 'специалитет'}), Document(page_content='Студенческие организации\\nЭкстра — бренд внеучебной жизни, объединяющий все сервисы, проекты и\\nпродукты Вышки, которые развивают и поддерживают творческие, научные и\\nпрофессиональные идеи студентов  [Вышка Добра](https:\/\/dobrovyshka.hse.ru)\\nОбучение служением, волонтерство и путешествия по России [Путь\\nпредпринимателя](https:\/\/inc.hse.ru) Акселераторы, инкубационные программы и\\nстартапы [ГосВышка](https:\/\/gosvyshka.hse.ru) Лидерство, развитие\\nуправленческих навыков, карьера [Республика\\nученых](https:\/\/studscience.hse.ru) Академическое развитие, поддержка\\nстуденческих научных проектов [Базовая платформа](https:\/\/t.me\/basedplatform)\\nТвой гид в мире внеучебки. Это медиа, которое рассказывает о тебе и твоих\\nоднокурсниках за пределами аудиторий, а также о возможностях, которыми ты\\nможешь пользоваться в Вышке. Читай новости и материалы о разных сферах\\nвнеучебки. Наука, спорт, социальные проекты, карьера, предпринимательство,\\nкультура и психологическое благополучие. Обо всем этом в наших социальных\\nсет', metadata={'download_url': 'https:\/\/extra.hse.ru\/#directions', 'file_name': 'Студенческие организации.docx', 'source': 'внешний сайт', 'topic_tag_1': 'Внеучебка', 'topic_tag_2': 'Другое', 'url': 'https:\/\/extra.hse.ru\/#directions', 'user_tag_1': 'Москва', 'user_tag_2': 'бакалавриат', 'user_tag_3': 'аспирантура', 'user_tag_4': 'магистратура', 'user_tag_5': 'специалитет'})]"""
question = "Как получить психологическую помощь?"

answer = generate_hse_answer(
    question=question,
    context=context
)

print(f"Вопрос: {question}\nОтвет: {answer}")

Вопрос: Как получить психологическую помощь?
Ответ: Я понял ваш вопрос о дне начала военной подготовки

Чтобы узнать конкретную дату начала военной подготовки во ВУЦ Нижнего Новгорода для вас, я проверил анкету заявления на обучение, представленную в системе поддержки процесса обучения и преподавания ННГУ https://eduapp.nnusr.com/ap/student/#/newApplicationInfoForm

С вашего адреса электронной почты мы узнали имя пользователя и пароль, необходимые для доступа в систему поддержки процесса обучения и преподавания ННГУ http://education.nnusr.

Чтобы найти занятия со ссылкой на пункт "Ссылка на электронную форму заявок", необходимо следовать инструкциям для создания учетной записи в системе поддержки процесса обучения и преподавания ННГУ https://eduapp.nnusr.com/ap/student/#createAccount
